In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

In [5]:
connString = "jdbc:mysql://mysql:3306/default?useUnicode=true&serverTimezone=Asia/Seoul"
tableName = "korean_movies"
accessInfo = {"user":"scott", "password":"tiger"}
korean_movies = spark.read.jdbc(connString, tableName, properties=accessInfo)

In [8]:
display(korean_movies)

timestamp,rate,movie,year,grade,title,main_actor,country,genre,time
2017-05-07 12:34:00,7,10003,2015,12세 관람가,빽 투 더 퓨쳐 2,마이클 J. 폭스,미국,SF,1494128040
2016-07-03 16:10:00,7,10004,1990,전체 관람가,빽 투 더 퓨쳐 3,마이클 J. 폭스,미국,서부,1467529800
2017-12-15 22:22:00,9,10018,2011,전체 관람가,이티,헨리 토마스,미국,판타지,1513344120
2015-02-21 14:53:00,9,10021,2017,12세 관람가,록키,실베스터 스탤론,미국,드라마,1424497980
2015-03-29 20:09:00,7,10022,1980,12세 관람가,록키 2,실베스터 스탤론,미국,드라마,1427627340
2015-04-11 16:48:00,7,10023,1982,12세 관람가,록키 3,실베스터 스탤론,미국,드라마,1428738480
2015-04-18 21:17:00,4,10024,1987,12세 관람가,록키 4,실베스터 스탤론,미국,드라마,1429359420
2015-06-06 20:43:00,4,10025,1990,15세 관람가,록키 5,실베스터 스탤론,미국,드라마,1433590980
2015-02-28 16:28:00,10,10028,1984,청소년 관람불가,터미네이터,아놀드 슈왈제네거,미국,SF,1425108480
2017-12-18 22:12:00,9,10029,1978,12세 관람가,죠스,로이 샤이더,미국,스릴러,1513602720


In [15]:
korean_movies.repartition(30).write.mode("overwrite").json("data/korean_movies")

In [16]:
json = spark.read.json("data/korean_movies")

In [17]:
json.printSchema()

root
 |-- country: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- main_actor: string (nullable = true)
 |-- movie: long (nullable = true)
 |-- rate: long (nullable = true)
 |-- time: long (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [18]:
display(json)

country,genre,grade,main_actor,movie,rate,time,timestamp,title,year
미국,드라마,12세 관람가,로빈 윌리엄스,10048,10,1132759560,2005-11-24 00:26:00,죽은 시인의 사회,2016
프랑스,드라마,청소년 관람불가,스티브 맥퀸,10046,10,1199927340,2008-01-10 10:09:00,빠삐용,2016
미국,SF,청소년 관람불가,아놀드 슈왈제네거,10028,7,1310220900,2011-07-09 23:15:00,터미네이터,1984
미국,SF,15세 관람가,톰 스커릿,10037,7,1290358920,2010-11-22 02:02:00,에이리언,1987
미국,판타지,15세 관람가,조니 뎁,10936,10,1150827540,2006-06-21 03:19:00,가위손,2014
미국,드라마,15세 관람가,진 핵크만,10041,10,1507042320,2017-10-03 23:52:00,포세이돈 어드벤쳐,1978
미국,드라마,12세 관람가,케빈 코스트너,10244,10,1318954980,2011-10-19 01:23:00,늑대와 춤을,1991
null,null,null,null,10707,10,1248243180,2009-07-22 15:13:00,null,null
프랑스,드라마,청소년 관람불가,스티브 맥퀸,10046,10,1568370900,2019-09-13 19:35:00,빠삐용,2016
한국,멜로/로맨스,12세 관람가,안성기,10764,10,1312641780,2011-08-06 23:43:00,기쁜 우리 젊은 날,1987
